In [1]:
# take video 5 as an example

In [2]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import time
from tqdm import trange 

def get_bilibili_url(start, end):
    url_list = []
    date_list = [i for i in pd.date_range(start, end).strftime('%Y-%m-%d')]
    for date in date_list:
        url = f"https://api.bilibili.com/x/v2/dm/history?type=1&oid=157525597&date={date}"
        url_list.append(url)
    return url_list

def get_bilibili_danmu(url_list):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
        "cookie": "_uuid=0B760BD5-AD42-0E3E-2A8E-383FE17DCCA217028infoc; buvid3=E644C7C2-3090-4E6B-BF4B-9EFFB6C46171143088infoc; CURRENT_FNVAL=80; blackside_state=1; rpdid=|(kRkmm~Yuum0J'uY|llm|u)R; DedeUserID=535114874; DedeUserID__ckMd5=f3dcf24274bdc507; SESSDATA=8a5069fc%2C1621149170%2Cfea31*b1; bili_jct=3c0681ff267a1bf41051b2894bc165d2; bsource=search_google; bfe_id=74d2c8da80807a7b17efa7e7578c6f68; sid=jm8e1lp8" #Headers中copy即可
               }
    
   
    file = open("bilibili_danmu.txt", 'w')
    for i in trange(len(url_list)):
        url = url_list[i]
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text)
        data = soup.find_all("d")
        danmu = [data[i].text for i in range(len(data))]
        for items in danmu:
            file.write(items)
            file.write("\n")
        time.sleep(3)
    file.close()


if __name__ == "__main__":
    start = '2/26/2020' 
    end = '2/28/2020' 
    url_list = get_bilibili_url(start, end)
    get_bilibili_danmu(url_list)
    print("bilibili 乾杯")

100%|██████████| 3/3 [00:09<00:00,  3.26s/it]

bilibili 乾杯


In [15]:
import csv
with open('/Users/tsubomikaji/Desktop/raw data/video1_danmaku.csv','w+',newline='', encoding='utf-8')as csvfile:
    spamriter=csv.writer(csvfile,dialect='excel')
    with open('/Users/tsubomikaji/Desktop/弹幕/清理data(2版/clean1.txt','r',encoding='utf-8')as filein:
        for line in filein:
            line_list=line.strip('\n').split(',')
            spamriter.writerow(line_list)

In [7]:
import numpy as np
import pandas as pd
import glob
import re
import jieba 

In [8]:
df = pd.read_csv("/Users/tsubomikaji/Desktop/弹幕/原始data/5.csv",header=None,error_bad_lines=False) 
df = df.iloc[:121442] #user and words
df = df.drop_duplicates() #duplicates
df = df.dropna() #losing words
df.columns = ["danmu"] #name
df
#delete repetitive words
def yasuo(st):
    for i in range(1,int(len(st)/2)+1):
        for j in range(len(st)):
            if st[j:j+2] == st[j+i:j+2*i]:
                k = j + i
                while st[k:k+i] == st[k+i:k+2*i] and k<len(st):   
                    k = k + i
                st = st[:j] + st[k:]    
    return st
df


b'Skipping line 1439: expected 1 fields, saw 2\nSkipping line 2315: expected 1 fields, saw 2\nSkipping line 2494: expected 1 fields, saw 2\n'


,danmu
0,⭐⭐⭐⭐⭐
1,暖心
2,害怕
3,十个泪目九个怒，还有一个在狂怒
4,怕了怕了
...,...
2988,借你吉言希望中考有个好成绩
2992,有点感动咋回事，先不给五星，给三星吧
2993,我钉钉响了
2994,小学生嘴给我干净点！


In [9]:
#delete repetitive words
def yasuo(st):
    for i in range(1,int(len(st)/2)+1):
        for j in range(len(st)):
            if st[j:j+2] == st[j+i:j+2*i]:
                k = j + i
                while st[k:k+i] == st[k+i:k+2*i] and k<len(st):   
                    k = k + i
                st = st[:j] + st[k:]    
    return st
yasuo(st="啊啊啊啊啊啊啊")

'啊啊啊'

In [10]:
df["danmu"] = df["danmu"].apply(yasuo)
df

,danmu
0,⭐⭐⭐
1,暖心
2,害怕
3,十个泪目九个怒，还有一个在狂怒
4,怕了
...,...
2988,借你吉言希望中考有个好成绩
2992,有点感动咋回事，先不给五星，给三星吧
2993,我钉钉响了
2994,小学生嘴给我干净点！


In [11]:
#special marks
df['danmu'] = df['danmu'].str.extract(r"([\u4e00-\u9fa5]+)")
df = df.dropna()
df

,danmu
1,暖心
2,害怕
3,十个泪目九个怒
4,怕了
5,加油
...,...
2988,借你吉言希望中考有个好成绩
2992,有点感动咋回事
2993,我钉钉响了
2994,小学生嘴给我干净点


In [12]:
#short words
df = df[df["danmu"].apply(len)>=4]
df = df.dropna()
df

,danmu
3,十个泪目九个怒
6,高考不留遗憾
7,高考加油
14,卧槽太真实了
17,借你吉言
...,...
2988,借你吉言希望中考有个好成绩
2992,有点感动咋回事
2993,我钉钉响了
2994,小学生嘴给我干净点


In [13]:
import csv

In [14]:
df.to_csv('/Users/tsubomikaji/Desktop/弹幕/原始data/clean5.csv')